In [1]:
import numpy
import cv2
import random
import pickle
import sklearn
import tensorflow as tf
import progressbar

In [2]:
from image_preprocessing.scan import *
from image_preprocessing.helper import resize,get_sorted_contours_bounding_box,skew_correction

In [3]:
img = cv2.imread(r'C:\Users\Avinash\Desktop\New folder\OCR\skew corrected.png')
try:
    img = document_warper(img)
except FourPointException:
    print("couldn't find the edges of the current doucment.\nAt present, to improve accuracy we are working on skew correctrion on the present document")
    img = skew_correction(img)

couldn't find the edges of the current doucment.
At present, to improve accuracy we are working on skew correctrion on the present document
Best angle: 0


In [4]:
print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

(756, 1538, 3)


In [5]:
edged = edge_detection(img)
cnts,heirarchy = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)


In [6]:
cnts,bounding_box = get_sorted_contours_bounding_box(cnts)
cv2.imshow('w',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
chars=[]
boxs=[]
for i,c in enumerate(cnts):
    x,y,w,h = bounding_box[i]
    if (w>=7 and w<=150) and (h>=15 and h<=150):
        roi = gray[y:y+h,x:x+w]
        thresh = cv2.threshold(roi,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        dims = random.randint(45,55)
        thresh = cv2.resize(thresh,(dims,dims),cv2.INTER_CUBIC)
        tH,tW = thresh.shape
        dX = int(max(0,128-tW)/2.0)
        dY = int(max(0,128-tH)/2.0)
        padded = cv2.copyMakeBorder(thresh,top=dY,bottom=dY,right=dX,left=dX,borderType=cv2.BORDER_CONSTANT,value=(255,255,255))
        padded = cv2.resize(padded,(128,128),cv2.INTER_CUBIC)
        padded = padded.astype('float32')/255.
        padded = np.expand_dims(padded,axis=-1)
        boxs.append((x,y,w,h))
        chars.append(padded)

In [8]:
from tensorflow.keras.models import load_model
import json

In [9]:

with tf.device('/cpu:0'):
    nums = load_model('models/alex_net nums.h5')
    lowercase = load_model('models/incep lower.h5')

In [10]:
with tf.device('/gpu:1'):
    uppercase = load_model('models/incep upper.h5')

In [26]:

with open('final rf 0.90.sav','rb') as f:
    random_forest = pickle.load(f)

In [12]:
chars = np.array(chars,dtype='float32')

In [13]:
with open('classes/lower_classes.json','r') as f:
    lower_classes = json.load(f)
f.close()
with open('classes/upper_classes.json','r') as f:
    upper_classes = json.load(f)
f.close()

In [14]:
int('30',16)

48

In [18]:
def rf_classify(rf_preds,uc,lc,nums,chars,bar):
    letter_class = []
    probs = []
    for i,c in enumerate(chars):
        tmp = np.reshape(c,((1,)+c.shape))

        if(rf_preds[i]==0):
            
            upper_pred = uppercase.predict(tmp)
            lower_pred = lowercase.predict(tmp)
            idx1,idx2 = np.argmax(upper_pred),np.argmax(lower_pred)
            final_class = chr(int(upper_classes[str(idx1)],16))
            prob = upper_pred[0][idx1]
            if(upper_pred[0][idx1]<lower_pred[0][idx2]):
                final_class = chr(int(lower_classes[str(idx2)],16))
                prob = lower_pred[0][idx2]
        if(rf_preds[i]==1 or prob<0.5):

            preds = nums.predict(tmp)
            idx1 = np.argmax(preds)
            final_class = chr(48+idx1)
            prob = preds[0][idx1]
        letter_class.append(final_class)
        probs.append(prob)
        bar.update(i)
    return letter_class,probs


In [16]:

rf_preds=random_forest.predict(chars.reshape((chars.shape[0],chars.shape[1]*chars.shape[2])))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 170 out of 170 | elapsed:    0.1s finished


In [25]:

  
widgets = ['Classifying images [',
         progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
         '] ',
           progressbar.Bar('='),' (',
           progressbar.ETA(), ') ',
          ]  
          

progressbar.streams.flush()
bar = progressbar.ProgressBar(max_value=chars.shape[0],
                              widgets=widgets).start()


labels,probs = rf_classify(rf_preds,uppercase,lowercase,nums,chars,bar)
bar.finish()
for (label,(x,y,w,h),prob) in zip(labels,boxs,probs):
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,label,(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,255,0),2)
    
    
cv2.imshow('final',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Classifying images [elapsed time: 0:02:45] |================| (Time:  0:02:45) 
Creating a overview images [elapsed time: 0:00:00] |========| (Time:  0:00:00) 


In [18]:
cv2.imwrite(r"C:\Users\Avinash\Desktop\New folder\OCR\test result\hello world.jpg",img)

True